## 简答题
1. 如果训练集有100万个实例，训练决策树（无约束）大致的深度是多少？
无约束会尽可能分裂log(100w)
2. 通常来说，子节点的基尼杂质是高于还是低于其父节点？是通常更高/更低？还是永远更高/更低？
子节点的基尼杂质是通常低于其父节点
3. 如果决策树过拟合训练集，减少max_depth是否为一个好主意？
是
4. 如果决策树对训练集欠拟合，尝试缩放输入特征是否为一个好主意？
决策树对缩放不敏感
5. 如果在给定的训练集上训练决策树需要一个小时，那么如果将特征数量变为两倍，训练大约需要多少时间？
决策树训练时间与特征数量d O(d*n*logn) 训练2h
6. 如果在包含100万个实例的训练集上训练决策树需要一个小时，那么在包含1000万个实例的训练集上训练决策树，大概需要多长时间？提示：考虑CART算法的计算复杂度。
决策树训练时间与特征数量d O(d*n*logn) 10^6*log(10^6)/10^7*log(10^7)≈11.5h

## 编程题

1. 为 新月形 数据集训练并微调一棵决策树。

a. 使用make_moons(n_samples=10000，noise=0.4)生成一个 新月形 数据集。

b. 使用train_test_split()拆分训练集和测试集。

c. 使用交叉验证的网格搜索（在GridSearchCV类的帮助下）为Decision-TreeClassifier找到适合的超参数值。提示：尝试max_leaf_nodes的多种值。

d. 使用超参数对整个训练集进行训练，并测量模型在测试集上的性能。你应该得到约85%～87%的准确率。

In [2]:
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeClassifier

X, y = make_moons(n_samples=10000, noise=0.4, random_state=42)
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)
param_grid={'max_leaf_nodes':[10,15,20]}
GSCV_tree = GridSearchCV(DecisionTreeClassifier(random_state=42), param_grid)
GSCV_tree.fit(X_train,y_train)
print(GSCV_tree.best_params_)
y_pred = GSCV_tree.predict(X_test)
print(GSCV_tree.score(X_test,y_test))

{'max_leaf_nodes': 20}
0.87


2. 按照以下步骤种植森林。

a.继续之前的练习，生产1000个训练集子集，每个子集包含随机挑选的100个实例。提示：使用Scikit-Learn的ShuffleSplit类来实现。

b.使用前面得到的最佳超参数值，在每个子集上训练一棵决策树。在测试集上评估这1000棵决策树。因为训练集更小，所以这些决策树的表现可能比第一棵决策树要差一些，只能达到约80%的精度。

c.见证奇迹的时刻到了。用每个测试集实例，生成1000棵决策树的预测，然后仅保留次数最频繁的预测［可以使用SciPy的mode()函数］。这样你在测试集上可获得大多数投票的预测结果。

d.评估测试集上的这些预测，你得到的准确率应该比第一个模型更高（高出0.5%～1.5%）。恭喜，你已经训练出了一个随机森林分类器！

In [20]:
from sklearn.model_selection import ShuffleSplit
from sklearn.tree import DecisionTreeClassifier
from scipy.stats import mode
from sklearn.metrics import accuracy_score
import numpy as np

shuffle_split = ShuffleSplit(n_splits=1000, train_size=100, random_state=42)
trees = []
scores = []
for train_index, _ in shuffle_split.split(X_train):
    subset_X = X_train[train_index]
    subset_y = y_train[train_index]
    tree = DecisionTreeClassifier(max_leaf_nodes=20, random_state=42)
    tree.fit(subset_X, subset_y)
    trees.append(tree)
    score = tree.score(X_test, y_test)
    scores.append(score)
print("mean_score",np.mean(scores))
all_predictions = np.array([tree.predict(X_test) for tree in trees])
majority_predictions,_ = mode(all_predictions, axis=0)
print("accuracy_score",accuracy_score(y_test, majority_predictions))


mean_score 0.8012284999999999
accuracy_score 0.872
